Git
how to choose companies
Time stamp
Feature, Target

words or sentence? -> libraries to calculate sentiment
predictprob function
lower - case

In [1]:
import pandas as pd

In [2]:
SP_df = pd.read_csv("WIKI_PRICES.csv")

In [3]:
SP_df.head()

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
0,A,1999-11-18,45.50,50.00,40.00,44.00,44739900.0,0.0,1.0,31.105117,34.181447,27.345157,30.079673,44739900.0
1,A,1999-11-19,42.94,43.00,39.81,40.38,10897100.0,0.0,1.0,29.355027,29.396044,27.215268,27.604936,10897100.0
2,A,1999-11-22,41.31,44.00,40.06,44.00,4705200.0,0.0,1.0,28.240711,30.079673,27.386175,30.079673,4705200.0
3,A,1999-11-23,42.50,43.63,40.25,40.25,4274400.0,0.0,1.0,29.054230,29.826730,27.516065,27.516065,4274400.0
4,A,1999-11-24,40.13,41.94,40.00,41.06,3464400.0,0.0,1.0,27.434029,28.671398,27.345157,28.069804,3464400.0


In [4]:
len(SP_df)

14935063

In [5]:
NEWS_df = pd.read_csv("uci-news-aggregator.csv")

In [6]:
NEWS_df.head()

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
2,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
3,4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
4,5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027


In [7]:
len(NEWS_df)

422419

In [8]:
len(NEWS_df[NEWS_df['TITLE'].str.contains("IBM", na=False)])

1058

In [10]:
companies = {"3M": "MMM",
             "American Express":"AXP",
             "Apple":"AAPL",
             "Boeing":"BA",
             "Caterpillar":"CAT",
             "Chevron":"CVX",
             "Cisco Systems":"CSCO",
             "Coca-Cola":"KO",
             "DuPont":"DD",
             "ExxonMobil":"XOM",
             "General Electric":"GE",
             "Goldman Sachs":"GS",
             "Home Depot":"HD",
             "IBM":"IBM",
             "Intel":"INTC",
             "Johnson & Johnson":"JNJ",
             "JPMorgan Chase":"JPM",
             "McDonald's":"MCD",
             "Merck":"MRK",
             "Microsoft":"MSFT",
             "Nike":"NKE",
             "Pfizer":"PFE",
             "Procter & Gamble":"PG",
             "Travelers":"TRV",
             "UnitedHealth Group":"UNH",
             "United Technologies":"UTX",
             "Verizon":"VZ",
             "Visa":"V",
             "Walmart":"WMT",
             "Walt Disney":"DIS"
}